In [1]:
"""
2023.04.18继续B站教程
conda:wenet
"""
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

/home/suncheng/anaconda3/envs/wenet/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.maxpool ceil_model

In [12]:
class MyMaxPool(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.maxpool = nn.MaxPool2d(kernel_size=3, ceil_mode=False)      # 注意参数ceil_model--取整
    
    def forward(self, x):
        x = self.maxpool(x)
        return x

In [10]:
input = torch.tensor(data=[
    [1, 2, 0, 3, 1],
    [0, 1, 2, 3, 1],
    [1, 2, 1, 0, 0],
    [5, 2, 3, 1, 1],
    [2, 1, 0, 1, 1]
], dtype=torch.float32)
input = input.reshape(shape=(-1, 1, 5, 5))      # 转为BCHW

In [13]:
mymaxpool = MyMaxPool()
output = mymaxpool(input)
output

tensor([[[[2.]]]])

In [17]:
dataset = datasets.CIFAR10('data', train=False, transform=transforms.ToTensor(), download=True)
dataloader = DataLoader(dataset=dataset, batch_size=64)

writer = SummaryWriter(log_dir='logs')

Files already downloaded and verified


2023-04-18 14:36:39.765038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 14:36:40.059759: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-18 14:36:40.911366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-04-18 14:36:40.911513: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot 

In [21]:
for i, data in enumerate(dataloader):
    images, labels = data
    writer.add_images('input', images, i)    # 构造dataloader时已经转为tensor了
    output = mymaxpool(images)
    writer.add_images('output', output, i)
writer.close()

2.Relu

In [31]:
class MyRelu(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x = self.relu(x)
        x = self.sigmoid(x)
        return x

In [25]:
input = torch.tensor([
    [1, -.5],
    [-1, 3]
])
output = input.reshape(shape=(-1, 1, 2, 2))
output.shape

torch.Size([1, 1, 2, 2])

In [32]:
myrelu = MyRelu()
output = myrelu(input)
output

tensor([[0.7311, 0.3775],
        [0.2689, 0.9526]])

In [33]:
# sigmoid对图片的处理效果
writer = SummaryWriter('logs')
for i, data in enumerate(dataloader):
    writer.add_images('sigmoid input', data[0], i)
    output = myrelu(data[0])
    writer.add_images('sigmoid output', output, i)
writer.close()

3.线性层

In [35]:
class MyLinear(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear = nn.Linear(196608, 10)     # 196608是将cifar10的图片flatten后的形状

    def forward(self, x):
        x = self.linear(x)
        return x
    
mylinear = MyLinear()

In [ ]:
for i, data in enumerate(dataloader):
    images, labels = data
    print("batch images: ", images.shape)
    # reshape相当于flatten，之后进网络
    # input = images.reshape((1, 1, 1, -1))   # 可用torch.flatten替换
    input = torch.flatten(images)
    print("batch reshape: ", input.shape)
    output = mylinear(input)
    print("linear: ", output.shape)

4.nn.Sequence

In [2]:
class MySequence(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.seq = nn.Sequential(*[
            nn.Conv2d(3, 32, kernel_size=5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, kernel_size=5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10)
        ])
    
    def forward(self, x):
        x = self.seq(x)
        return x

In [3]:
myseq = MySequence()

input = torch.zeros(size=(64, 3, 32, 32))
output = myseq(input)
output.shape

torch.Size([64, 10])

In [9]:
writer = SummaryWriter(log_dir='logs')
writer.add_graph(myseq, input)
writer.close()

2023-04-18 19:27:17.049377: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 19:27:17.372605: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-18 19:27:18.167269: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-04-18 19:27:18.167398: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot 

5.Loss

In [14]:
input = torch.tensor([1, 2, 3], dtype=torch.float32)
output = torch.tensor([1, 2, 5], dtype=torch.float32)
loss = nn.L1Loss(reduction='sum')
loss_mse = nn.MSELoss(reduction='sum')
loss(input, output), loss_mse(input, output)

(tensor(2.), tensor(4.))

In [4]:
# 交叉熵损失
x = torch.tensor([.1, .2, .3])
y = torch.tensor([1])
loss_ce = nn.CrossEntropyLoss()
loss_ce(x.reshape(shape=(1, 3)), y)     # 交叉熵要求输入是 N Class_num

tensor(1.1019)

In [5]:
dataset = datasets.CIFAR10('data', train=False, transform=transforms.ToTensor(), download=True)
dataloader = DataLoader(dataset=dataset, batch_size=64)

# 之前已经定义了loss，这里再定义optim
optimizer = torch.optim.SGD(myseq.parameters(), lr=.05)
for epoch in range(20):
    running_loss = 0    # 一次完整训练，在所有数据样本上的loss之和
    for data in dataloader:     # 只循环这一次，相当于只看一遍数据，并不能学到对这些数据的预测能力，通常要学好几遍
        imgs, labels = data
        output = myseq(imgs)    # myseq=MySequence()
        # print(output, labels)   # 分类问题就用交叉熵
        l = loss_ce(output, labels)
        # print(f'epoch:{epoch + 1} loss:{l:.7f}')

        optimizer.zero_grad()   # 梯度置0
        l.backward()    # 计算网络中权重的梯度，后面才能以此更新权重  反向传播才能计算梯度
        optimizer.step()

        running_loss += l
    print(f'------------------> epoch:{epoch + 1} running loss:{running_loss}')

Files already downloaded and verified
------------------> epoch:1 running loss:345.34613037109375
------------------> epoch:2 running loss:301.598876953125
------------------> epoch:3 running loss:273.929443359375
------------------> epoch:4 running loss:254.6281280517578
------------------> epoch:5 running loss:240.33053588867188
------------------> epoch:6 running loss:227.9118194580078
------------------> epoch:7 running loss:215.84518432617188
------------------> epoch:8 running loss:204.2844696044922
------------------> epoch:9 running loss:193.0252227783203
------------------> epoch:10 running loss:182.3740234375
------------------> epoch:11 running loss:172.07681274414062
------------------> epoch:12 running loss:161.62889099121094
------------------> epoch:13 running loss:150.94503784179688
------------------> epoch:14 running loss:139.67620849609375
------------------> epoch:15 running loss:128.417724609375
------------------> epoch:16 running loss:116.79631042480469
---------

6.Pytorch内置模型

In [6]:
import torchvision

vgg16 = torchvision.models.vgg16_bn(pretrained=True, progress=True)

In [9]:
vgg16.classifier.add_module('add_linear', nn.Linear(1000, 10))      # 添加1个线性层
vgg16.classifier[6] = nn.Linear(1000, 10)   # 修改1个线性层
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256